# Project 2: Gaussian Elimination and Least Squares

**Your name here:**    <i>Kevin Wong</i>

## Computing power
The LINPACK Benchmarks are a measure of a system's floating point computing power. Introduced by Jack Dongarra, they measure how fast a computer solves a dense n by n system of linear equations $Ax = b$. This is the standard measure for computer speed. Supercomputers (which usually have linux operating systems) can do about 10 petaflops in one second. One petaflop is $10^{15}$ flops. 

Titan, a supercomputer in the Oak Ridge National lab, is the fastest computer in the US. It can perform 17.59 petaflops every second.

(2) Calculate how much time (in seconds) does Titan need to solve a 5000 by 5000 system $Ax=b$, assuming it takes $\frac{2}{3}n^3$ flops for Gauss Elimination?

In [21]:
from __future__ import division
import numpy as np

In [489]:
# Number of floating point operations needed to solve a 5000 x 5000 system
flops = (2/3)*(5000**3)
flops

83333333333.33333

In [490]:
# Speed is seconds / flop
speed = (1/17.59e15)
print "Titan needs " + str(flops * speed) + " seconds to solve this system."

Titan needs 4.73754026909e-06 seconds to solve this system.


Laptops nowadays is around 10's gigaflops. **Now let's see how fast your own machine is**.

(3) Create a 5000 by 5000 random matrix $R$, and create a 5000 by 1 random vector $rb$, where each entry of $R$ and $rb$ is standard normal distribution (Hint: look up numpy.random)

In [32]:
matrix_R = np.random.random_sample((5000, 5000))
matrix_R

array([[ 0.20297266,  0.44583828,  0.71365124, ...,  0.36752773,
         0.89664547,  0.77969705],
       [ 0.56848622,  0.29928423,  0.92575022, ...,  0.89474379,
         0.67847516,  0.80210088],
       [ 0.99310362,  0.76669324,  0.14551459, ...,  0.70357006,
         0.87447339,  0.89038864],
       ..., 
       [ 0.46671164,  0.07044469,  0.46179481, ...,  0.19961043,
         0.23365951,  0.46062463],
       [ 0.29646141,  0.54859435,  0.81998432, ...,  0.49982208,
         0.40516469,  0.02266658],
       [ 0.43993653,  0.45650413,  0.02144098, ...,  0.95532874,
         0.3769568 ,  0.67178049]])

In [33]:
vector_rb = np.random.random_sample((5000, 1))
vector_rb

array([[ 0.3615245 ],
       [ 0.34885254],
       [ 0.12914578],
       ..., 
       [ 0.12375889],
       [ 0.05551473],
       [ 0.1944551 ]])

(4) Measure how much time does it take to solve this random $Rx=rb$. Below is an easy way to measure time

In [50]:
import timeit
start = timeit.default_timer()

# Solving the linear equation for x.
np.linalg.solve(matrix_R, vector_rb)

end = timeit.default_timer()
time = end - start
print  "Solving took " + str(time) + " seconds"

Solving took 2.28144288063 seconds


(5) Now calculate how many gigaflops your computer can do in a second.

In [51]:
# Gigaflop is 10**9 flops

# flops / sec   *   gigaflops / flops   =  gigflops / sec
print "My computer solved this problem at " + str((flops / time) / (1e9)) + " gigaflops per second"

My computer solved this problem at 36.5265920268 gigaflops per second


(5) Also measure how much total time your computer takes to find inverse of $R$ and multiply $R^{-1}*b$. You should get the message that this is a worse way to solve $Rx=rb$. In general, it never does you any good to find inverse in numerical computing.

In [48]:
 start = timeit.default_timer()

# Solving by finding the inverse and multiplying
matrix_R_inv = np.linalg.inv(matrix_R)
np.linalg.multi_dot((matrix_R_inv, vector_rb))

end = timeit.default_timer()
time = end - start
print  "Solving by finding the inverse took " + str(time) + " seconds"

Solving by finding the inverse took 8.19817996025 seconds


## Gaussian Elimination from scratch

Below is the SPLU function that I wrote. It decompose $A$ as $A=PLU$ using **Scaled Partial Pivoting**, where $P$ is a permutation matrix, $L$ is lower triangular, and $U$ is upper triangular.

In [56]:
import numpy as np
# Input A is any invertible matrix
# return P,L,U such that A = PLU
def SPLU(A):     
    sA = A.shape
    # make sure it is in float
    A = A.astype(float)
    
    #Check A is square    
    if sA[0] != sA[1]:
        raise ValueError('You need to enter a square matrix.')
    n = sA[0]    
    # find the max in each row
    AA = np.absolute(A)
    s = np.amax(AA,axis=1)
    RowInd = []  #Initiate an empty list for row indices
    L = np.identity(n)
    for j in range(n-1): # at column j
        # Find the row index of the pivot
        allRI = list(set(range(n)) - set(RowInd))
        allRelV = AA[allRI,j]/s[allRI]
        fi = np.argmax(allRelV)
        ri = allRI[fi] #ri is the pivot row idex        
        #update RowInd
        RowInd.append(ri)        
        for i in list(set(allRI) - set([ri])): 
            if A[ri,j] == 0:
                raise ValueError('You need to enter an invertible matrix.')
            c = -A[i,j]/A[ri,j]
            A[i,j:n] = A[ri, j:n]*c + A[i,j:n] 
            if c != 0:
                L[i,ri]= -c
        print A
    # build P,L,U according to RowInd
    LastL = list(set(range(n)) - set(RowInd))
    LastI = LastL[0]
    RowInd.append(LastI)
    I = np.identity(n)
    P = I[:,RowInd]
    U = A[RowInd,:]
    L = L[:,RowInd] #permute columns 
    L = L[RowInd,:]
    return (P, L, U)

(6) Use the above function to check your work on chapter 4 exercise 6. Print out P,L,U, and make sure their product is the original matrix.

In [57]:
matrix_A = np.array(([3, -6, -3], [2, 0, 6], [-4, 7, 4]))
matrix_A

array([[ 3, -6, -3],
       [ 2,  0,  6],
       [-4,  7,  4]])

In [59]:
P, L, U = SPLU(matrix_A)

[[ 0.   -0.75  0.  ]
 [ 0.    3.5   8.  ]
 [-4.    7.    4.  ]]
[[ 0.   -0.75  0.  ]
 [ 0.    0.    8.  ]
 [-4.    7.    4.  ]]


In [60]:
P

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

In [61]:
L

array([[ 1.        ,  0.        ,  0.        ],
       [-0.75      ,  1.        ,  0.        ],
       [-0.5       , -4.66666667,  1.        ]])

In [62]:
U

array([[-4.  ,  7.  ,  4.  ],
       [ 0.  , -0.75,  0.  ],
       [ 0.  ,  0.  ,  8.  ]])

In [491]:
# Decomposition into permutation, lower triangular, and upper triangular matrices works
np.linalg.multi_dot([P, L, U])

array([[ 3., -6., -3.],
       [ 2.,  0.,  6.],
       [-4.,  7.,  4.]])

(7) Write a function SL(L,b) to solve $Lx=b$ where $L$ is lower triangular. Make sure you first verify that $L$ is lower triangular and invertible. 

**Try to write comments/explanations when you write a function.**

In [434]:
def SL(L, b):
    # Check L is invertible
    if np.linalg.det(L) == 0:
        raise ValueError('You need an invertible matrix')
        
    # Check L is lower triangular
    sL = L.shape
    row = sL[0]
    col = sL[1]    
    
    for i in range(1, col+1):
        if any(L[i-1, i:] != 0):
            raise ValueError("You need a lower triangular matrix")
            
    
    # Initialize solution to be returned
    solution_y = np.empty(row)
    
    # Increment through the rows starting at row 0
    i = 0
    while(i < row):
        
        # The first solution is found through simple division
        solution_y[i] = b[i] / L[i, 0]
        
        # The remaining terms must be found by incorporating previous "solved" terms
        if i > 0:
            right_side_term = b[i] 
            for j in range(i):
                multiplication = L[i, j]*solution_y[j]
                right_side_term = right_side_term - (multiplication)
                
            # Peform a "final" division to find each subsequent term after some algebra
            solution_y[i] = right_side_term / L[i, i]
            
        # Proceed to the next row
        i+=1
     
    return solution_y
    

(8) Test your SL function on chapter 4 exercise 5, where L is the lower triangular matrix there, and b=(-3,-22,3).

**Call the solution y**

In [492]:
matrix_L = np.array(([3, 0, 0], [2, 4, 0], [-4, -1, 2]))
vector_b = np.array([-3, -22, 3])

# Correctly solves for y
solution_y = SL(matrix_L, vector_b)
solution_y

array([-1., -5., -3.])

(9) Write a function SU(U,b) to solve $Ux=b$ where $U$ is upper triangular. Make sure you verify that $U$ is upper triangular and invertible. This should be only a small modification of your previous function.

In [436]:
def SU(U, b):
    # Check U is invertible
    if np.linalg.det(U) == 0:
        raise ValueError('You need an invertible matrix')
        
    # Check U is lower triangular
    sU = U.shape
    row = sU[0]
    col = sU[1]    
        
    for i in range(1, col):
        if any(U[i, :i] != 0):
            raise ValueError("You need an upper triangular matrix")
            
    
    # Initialize solution to be returned
    solution_x = np.empty(row)
    
    # Since we are working with an upper triangular, it makes sense to start at the bottom and work up
    i = row - 1
    while(i >= 0):
        
        # Calculate the last term first, through simple division
        solution_x[i] = b[i] / U[i, i]
        
        # For subsequent terms, incorporate previously solved terms to find a solution.
        if i < row - 1:
            right_side_term = b[i] 
            for j in range(col-1, i, -1):
                multiplication = U[i, j]*solution_x[j]
                right_side_term = right_side_term - (multiplication)
            solution_x[i] = right_side_term / U[i, i]
        
        # Proceed to the next row
        i-=1
     
    return solution_x
    

(10) Test your SU function on chapter 4 exercise 5, where U is the upper triangular matrix there, and b is the y vector that you solved in (8).

In [437]:
matrix_U = np.array(([1, -2, -1], [0, 1, 2], [0, 0, 1]))
matrix_U

array([[ 1, -2, -1],
       [ 0,  1,  2],
       [ 0,  0,  1]])

In [438]:
solution_y

array([-1., -5., -3.])

In [493]:
# Correctly solves for x
SU(matrix_U, solution_y)

array([-2.,  1., -3.])

(11) Write a function MySolve(A,b) that solves Ax=b using scaled partial pivoting. Your code will call SPLU, SU, SL, and should be only a few lines.

In [495]:
# Combines PLU decomposition with LU solving function to solve Ax = b
def MySolve(A, b):
    P, L, U = SPLU(A)
    b = np.linalg.multi_dot(([np.linalg.inv(P), b]))
    y = SL(L, b)
    return SU(U, y)

(12) Test MySolve on chapter 4 exercise 5.

In [496]:
matrix_A

array([[ 3, -6, -3],
       [ 2,  0,  6],
       [-4,  7,  4]])

In [497]:
vector_b

array([ -3, -22,   3])

In [499]:
# Correctly finds the weights of columns of A to give vector b
MySolve(matrix_A, vector_b)

[[ 0.   -0.75  0.  ]
 [ 0.    3.5   8.  ]
 [-4.    7.    4.  ]]
[[ 0.   -0.75  0.  ]
 [ 0.    0.    8.  ]
 [-4.    7.    4.  ]]


array([-2.,  1., -3.])

## Least Square via Cholesky

(13) Write a function LS(A,b) to find the least square solution of Ax=b, where A has independent columns. Your algorithm will first use python's cholesky function, then use your own SL and SU functions to find the final solution.

In [500]:
# Function to find the least squares solution x_ls via Cholesky factorization 
def LS(A, b):
    matrix_B = np.dot(A.T, A)
    f = np.dot(A.T, b)
    L = np.linalg.cholesky(matrix_B)
    y = SL(L, f)
    x_ls = SU(L.T, y)
    return x_ls

Below is the scores of games played between Team A, B, C, D, E

| Home | Away | Home score | Away score |
|:--- | :---  | :---  | :---  |
|A|B|10|4|
|A|C|1|3|
|A|D|8|4|
|A|E|6|4|
|B|A|5|9|
|B|C|1|13|
|B|D|5|4|
|C|A|16|12|
|C|B|11|13|
|C|D|8|4|
|C|E|9|3|
|D|A|8|13|
|D|B|5|4|
|D|C|6|12|
|E|B|3|4|
|E|C|1|3|
|E|D|3|6|

(14) Extra credit problem: Try your best to rank these 5 teams simply by staring at the table:

In [501]:
# Teams A and C do well, and in the games A plays against C, C won both times. 

# 1. C
# 2. A

# E lost all their games, so it's the worst team.

# 5. E

# B lost to A twice, and lost to C once and beat C once, but it was close. B won a few more times than D.

# 3. B
# 4. D

(15) Assign each team an overall score $s_i$ and use the function you wrote in (13) to find the scores. We can assume $\sum s_i=100$.

In [502]:
scores = np.array([[1, -1, 0, 0, 0],  # A
                   [-1, 0, 1, 0, 0], 
                   [1, 0, 0, -1, 0], 
                   [1, 0, 0, 0, -1], 
                   [1, -1, 0, 0, 0],  # B
                   [0, -1, 1, 0, 0],  
                   [0, 1, 0, -1, 0], 
                   [-1, 0, 1, 0, 0],  # C
                   [0, 1, -1, 0, 0], 
                   [0, 0, 1, -1, 0], 
                   [0, 0, 1, 0, -1], 
                   [1, 0, 0, -1, 0],  # D
                   [0, -1, 0, 1, 0], 
                   [0, 0, 1, -1, 0],
                   [0, 1, 0, 0, -1],  # E
                   [0, 0, 1, 0, -1], 
                   [0, 0, 0, 1, -1]])

score_differential = np.array([4, 3, 4, 4, 9, 13, 4, 12, 13, 4, 3, 13, 4, 12, 4, 3, 6])

In [503]:
# Call LS to find the least square solution to this linear system
x_ls = LS(scores, score_differential)
x_ls

array([ 2.72025463, -0.39085648,  4.00729167, -2.39085648, -2.409375  ])

In [504]:
# Adjust team weights Si such that the sum of the weights is 100.
sum(x_ls+19.6929)

100.00095833333333

In [505]:
# These are the adjusted team scores
x_ls = x_ls + 19.6929
x_ls

array([ 22.41315463,  19.30204352,  23.70019167,  17.30204352,  17.283525  ])

(16) Rank these 5 teams according to your solution:

In [506]:
# 1. Team C
# 2. Team A
# 3. Team B
# 4. Team D
# 5. Team E